In [17]:
import pennylane as qml
from pennylane import numpy as np
from qiskit.providers.fake_provider import *
from pennylane.transforms import mitigate_with_zne, richardson_extrapolate, fold_global

# the noisy simulator I'm using simulates the IBM Manila QC
def configured_backend():
    return FakeManila()

def ansatz(params, wires):
    qml.RY(params[0], wires=wires)

# controlled application of unitary matrix A. In this specific example I'm setting A to be the identity
def CA():
    # qml.CZ(wires=(1, 0))
    pass

def U_b():
    qml.Hadamard(wires=0)

# define devices
dev_noisy = qml.device("qiskit.remote", wires = 2, backend=configured_backend())
dev = qml.device("default.qubit", wires = 2)

# the main circuit (hadamard test)
def circuit(params, part=None, j=-1):
    # hadamard on the ancilla
    qml.Hadamard(wires=1)

    if part == "Im": # in this example we don't need this
        qml.PhaseShift(-np.pi/2, wires=1)

    ansatz(params, wires=0)
    # CA()
    U_b() # H

    if j != -1: # j = 0, (when we call it below), so this should induce a π phase on the |11> component
        qml.CZ(wires=[1, j])

    U_b() # H
    # CA()

    qml.Hadamard(wires=1)

    return qml.probs()

    # return qml.expval(qml.PauliZ(wires=1))

circuit_noiseless = qml.QNode(circuit, dev)
circuit_noisy = qml.QNode(circuit, dev_noisy)
circuit_mitiq = mitigate_with_zne(circuit_noisy, [1, 2, 3], fold_global, richardson_extrapolate)

for i in range(5):
    print(f"it {i}")
    print("noiseless probs: ", circuit_noiseless([np.pi/2], part="Re", j=0))
    print("noisy probs: ", circuit_noisy([np.pi/2], part="Re", j=0))
    print("mitiq probs: ", circuit_mitiq([np.pi/2], part="Re", j=0))

it 0
noiseless probs:  [5.00000000e-01 3.08148791e-33 5.00000000e-01 3.08148791e-33]
noisy probs:  [0.58398438 0.00878906 0.40234375 0.00488281]
mitiq probs:  (0.23144531249993736, 0.444335937500095, 0.05566406249994298, 0.2685546875000598)
it 1
noiseless probs:  [5.00000000e-01 3.08148791e-33 5.00000000e-01 3.08148791e-33]
noisy probs:  [0.56152344 0.00292969 0.4296875  0.00585938]
mitiq probs:  (0.2041015624999355, 0.38183593750008543, 0.15429687499995637, 0.25976562500005757)
it 2
noiseless probs:  [5.00000000e-01 3.08148791e-33 5.00000000e-01 3.08148791e-33]
noisy probs:  [0.54003906 0.01269531 0.44042969 0.00683594]
mitiq probs:  (0.04980468749991402, 0.45996093750009653, 0.21191406249996458, 0.2783203125000598)
it 3
noiseless probs:  [5.00000000e-01 3.08148791e-33 5.00000000e-01 3.08148791e-33]
noisy probs:  [0.55859375 0.00976562 0.421875   0.00976562]
mitiq probs:  (0.10253906249992563, 0.4511718750000964, 0.18945312499995606, 0.2568359375000568)
it 4
noiseless probs:  [5.00000

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from qiskit.providers.fake_provider import *
from pennylane.transforms import mitigate_with_zne, richardson_extrapolate, fold_global

# the noisy simulator I'm using simulates the IBM Manila QC
def configured_backend():
    return FakeManila()

def ansatz(params, wires):
    qml.RY(params[0], wires=wires)

# controlled application of unitary matrix A. In this specific example I'm setting A to be the identity
def CA():
    # qml.CZ(wires=(1, 0))
    pass

def U_b():
    qml.Hadamard(wires=0)

# define devices
dev_noisy = qml.device("qiskit.remote", wires = 2, backend=configured_backend())
dev = qml.device("default.qubit", wires = 2)

# the main circuit (hadamard test)
def circuit(params, part=None, j=-1):
    # hadamard on the ancilla
    qml.Hadamard(wires=1)

    if part == "Im":
        qml.PhaseShift(-np.pi/2, wires=1)

    ansatz(params, wires=0)
    # CA()
    U_b()

    if j != -1:
        qml.CZ(wires=[1, j])

    U_b()
    # CA()

    qml.Hadamard(wires=1)

    return qml.expval(qml.PauliZ(wires=1))

circuit_noiseless = qml.QNode(circuit, dev)
circuit_noisy = qml.QNode(circuit, dev_noisy)
circuit_mitiq = mitigate_with_zne(circuit_noisy, [1, 2, 3], fold_global, richardson_extrapolate)


def mu(params, j):
    mu_real = circuit_mitiq(params, part="Re", j=j)
    mu_imag = circuit_mitiq(params, part="Im", j=j)
    
    return mu_real + 1.0j * mu_imag

def cost_fun(params):
    mu_sum = abs(mu(params, 0))
    norm = abs(mu(params, -1))

    res = 0.5 - 0.5 * mu_sum / (1 * norm)

    return res

print(circuit_noiseless([np.pi/2], part="Re", j=0))
print(circuit_noisy([np.pi/2], part="Re", j=0))
print(circuit_mitiq([np.pi/2], part="Re", j=0))

0.9999999999999992
0.966796875
-0.32226562500025696


I got paranoid so I want to test if the cost function even spits out the correct output. Let
$$
I\ket{x} = \ket{+}
$$
be the system I'm trying to solve. Then for $V(\alpha) = R_y(\pi/2)$, I should get $C_L = 0$ in a noiseless simulator.

In [20]:
lr = 0.01
opt = qml.GradientDescentOptimizer(lr)
from IPython.display import clear_output
w = np.random.randn(2, 
requires_grad=True)
cost_history = []

for it in range(300):
    w, cost = opt.step_and_cost(cost_fun, w)
    clear_output(wait=True)
    print("Step {:3d}       Cost_L = {:9.7f}".format(it, cost), flush=True)
    cost_history.append(cost)

import matplotlib.pyplot as plt
plt.style.use("seaborn")
# plt.plot(np.log(cost_history), "g")
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()

Step   1       Cost_L = 0.0418379


KeyboardInterrupt: 

In [14]:
qml.about()

Name: PennyLane
Version: 0.33.1
Summary: PennyLane is a Python quantum machine learning library by Xanadu Inc.
Home-page: https://github.com/PennyLaneAI/pennylane
Author: 
Author-email: 
License: Apache License 2.0
Location: /Users/bigsad/Downloads/Algorithm-Research/Student-Hub/Indy-Ng/.venv/lib/python3.11/site-packages
Requires: appdirs, autograd, autoray, cachetools, networkx, numpy, pennylane-lightning, requests, rustworkx, scipy, semantic-version, toml, typing-extensions
Required-by: PennyLane-Lightning, PennyLane-qiskit

Platform info:           macOS-12.6-x86_64-i386-64bit
Python version:          3.11.6
Numpy version:           1.23.5
Scipy version:           1.10.1
Installed devices:
- default.gaussian (PennyLane-0.33.1)
- default.mixed (PennyLane-0.33.1)
- default.qubit (PennyLane-0.33.1)
- default.qubit.autograd (PennyLane-0.33.1)
- default.qubit.jax (PennyLane-0.33.1)
- default.qubit.legacy (PennyLane-0.33.1)
- default.qubit.tf (PennyLane-0.33.1)
- default.qubit.torch (Penn

In [2]:
dev3 = qml.device("default.qubit", wires = 2)
dev_noisy = qml.device("qiskit.remote", wires = 2, backend=configured_backend())

def ansatz(params, wires):
    qml.RY(params[0], wires=wires)

def U_b(wires):
    qml.Hadamard(wires=wires)

def test_hadamard(params, part="Re", j=-1):
    qml.Hadamard(wires=1)

    # qml.RY(params[0], 0)
    ansatz(params, wires=0)

    if part == "Im":
        qml.PhaseShift(-np.pi/2, wires=1)

    # qml.Hadamard(wires=0)
    U_b(wires=0)

    if j != -1:
        qml.CZ(wires=(1,j))


    # qml.Hadamard(wires=0)
    U_b(wires=0)

    qml.Hadamard(wires=1)

    # return qml.state()
    return qml.expval(qml.PauliZ(1))

In [4]:
test_hadamard_noiseless = qml.QNode(test_hadamard, dev3)
test_hadamard_noisy = qml.QNode(test_hadamard, dev_noisy)
test_hadamard_mitiq = mitigate_with_zne(test_hadamard, [1, 2, 3], fold_global, richardson_extrapolate)

In [5]:
# mu
print(test_hadamard_noiseless([np.pi/2], part="Re", j=0))
print(test_hadamard_noiseless([np.pi/2], part="Im", j=0))

#beta 
print(test_hadamard_noiseless([np.pi/2], part="Re", j=-1))
print(test_hadamard_noiseless([np.pi/2], part="Im", j=-1))

0.9999999999999992
0.0
0.9999999999999993
0.0


#### Try now with noise

In [6]:
# mu
print(test_hadamard_noisy([np.pi/2], part="Re", j=0))
print(test_hadamard_noisy([np.pi/2], part="Im", j=0))
#beta 
print(test_hadamard_noisy([np.pi/2], part="Re", j=-1))
print(test_hadamard_noisy([np.pi/2], part="Im", j=-1))

0.9609375
0.05859375
0.978515625
0.013671875


#### With Mitiq

In [8]:
# mu
print(test_hadamard_mitiq([np.pi/2], part="Re", j=0))
print(test_hadamard_mitiq([np.pi/2], part="Im", j=0))
#beta 
print(test_hadamard_mitiq([np.pi/2], part="Re", j=-1))
print(test_hadamard_mitiq([np.pi/2], part="Im", j=-1))

TransformError: Impossible to dispatch your transform on quantum function, because more than one tape is returned